# Projeto: Classificação de texto - avaliação de filmes (Comparar com IA)
## Alunos:
## David Wilson Melo Santos
## Miguel de Paula da Silva Cruz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import math
from struct import *
import matplotlib.pyplot as plt
import time
import os
import numpy

In [ ]:
def salvarArquivo(input_file, compressed_data, file):
  # storing the compressed string into a file (byte-wise).
    out = input_file.split(".")[0]
    output_file = open(out + ".lzw", "wb")
    for data in compressed_data:
        output_file.write(pack('>I',int(data)))

    output_file.close()
    file.close()

# Funções

In [ ]:
# Projeto: Classificação de texto - avaliação de filmes(Comparar com IA)
# Alunos:
#   David Wilson Melo Santos
#   Miguel de Paula da Silva Cruz

def comprimir(input_file, dictionary):
    dictionary_size = len(dictionary)
    n = 21
    maximum_table_size = pow(2,int(n)) + dictionary_size
    file = open(input_file, 'r', encoding='latin-1')
    data = file.read()

    # Building and initializing the dictionary.
    string = ""             # String is null.
    compressed_data = []    # variable to store the compressed data.

    # iterating through the input symbols.
    # LZW Compression algorithm
    for symbol in data:
        string_plus_symbol = string + symbol # get input symbol.
        if string_plus_symbol in dictionary:
            string = string_plus_symbol
        else:
            compressed_data.append(dictionary[string])
            if(len(dictionary) <= maximum_table_size):
                dictionary[string_plus_symbol] = dictionary_size
                dictionary_size += 1
            string = symbol
    if string in dictionary:
        compressed_data.append(dictionary[string])

def abrir_arquivos_em_ordem(diretorio, dictionary):
    # Listar os arquivos no diretório
    arquivos = os.listdir(diretorio)

    # Filtrar apenas os arquivos que seguem o padrão 0_x até 12499_x
    arquivos_filtrados = [arquivo for arquivo in arquivos if arquivo.startswith(tuple(str(i) + '_' for i in range(12500)))]

    # Ordenar os arquivos
    arquivos_ordenados = sorted(arquivos_filtrados, key=lambda x: int(x.split('_')[0]))

    # Contador para contar os arquivos abertos com sucesso
    contador_arquivos = 0

    # Abrir os arquivos em ordem crescente
    for arquivo in arquivos_ordenados:
        try:
            with open(os.path.join(diretorio, arquivo), 'r') as f:
                # Chama a função comprimir() passando o caminho do arquivo aberto como argumento
                comprimir(os.path.join(diretorio, arquivo), dictionary)
                # Ler conteúdo
                # conteudo = f.read()
                # print(f"Arquivo {arquivo}: {conteudo}")
                contador_arquivos += 1  # Incrementa o contador de arquivos abertos com sucesso
        except FileNotFoundError:
            print(f"Erro: Arquivo {arquivo} não encontrado.")

    # Printa o número de arquivos abertos com sucesso
    print(f"Total de arquivos abertos com sucesso: {contador_arquivos}")

def comprimir_estatico(input_file, dictionary):
    n=21 #Bits do dicionário 2^n
    file = open(input_file, 'r', encoding='latin-1')
    data = file.read()
    total_original_length = len(data)

    # Building and initializing the dictionary.
    string = ""             # String is null.
    compressed_data = []    # variable to store the compressed data.

    # iterating through the input symbols.
    # LZW Compression algorithm
    for symbol in data:
        string_plus_symbol = string + symbol # get input symbol.
        if string_plus_symbol in dictionary:
            string = string_plus_symbol
        else:
            compressed_data.append(dictionary[string])
            string = symbol
    if string in dictionary:
        compressed_data.append(dictionary[string])

    symbol_counts = {}
    for symbol in compressed_data:
        if symbol in symbol_counts:
            symbol_counts[symbol] += 1
        else:
            symbol_counts[symbol] = 1

    probabilities = [count / total_original_length for count in symbol_counts.values()]
    bits_per_symbol = sum(p*n for p in probabilities)
    return(bits_per_symbol)

def abrir_arquivos_em_ordem_teste(negative_dictionary, positive_dictionary, diretorio, saida, erro, acerto):
    # Listar os arquivos no diretório
    arquivos = os.listdir(diretorio)

    # Filtrar apenas os arquivos que seguem o padrão 0_x até 12499_x
    arquivos_filtrados = [arquivo for arquivo in arquivos if arquivo.startswith(tuple(str(i) + '_' for i in range(12500)))]

    # Ordenar os arquivos
    arquivos_ordenados = sorted(arquivos_filtrados, key=lambda x: int(x.split('_')[0]))

    # Contador para contar os arquivos abertos com sucesso
    contador_arquivos = 0

    # Abrir os arquivos em ordem crescente
    for arquivo in arquivos_ordenados:
        try:
            with open(os.path.join(diretorio, arquivo), 'r') as f:
                saida_negativa = comprimir_estatico(os.path.join(diretorio, arquivo), negative_dictionary)
                saida_positiva = comprimir_estatico(os.path.join(diretorio, arquivo), positive_dictionary)
                # conteudo = f.read()
                # print(f"Arquivo {arquivo}: {conteudo}")
                contador_arquivos += 1  # Incrementa o contador de arquivos abertos com sucesso
                if(saida_negativa > saida_positiva):
                  resultado = "positivo"
                else:
                  resultado = "negativo"

                if(resultado != saida):
                  erro += 1
                else:
                  acerto += 1

        except FileNotFoundError:
            print(f"Erro: Arquivo {arquivo} não encontrado.")

    # Printa o número de arquivos abertos com sucesso
    print(f"Total de arquivos abertos com sucesso: {contador_arquivos}")
    return erro, acerto

In [ ]:
dictionary_size = 256
negative_dictionary = {chr(i): i for i in range(dictionary_size)}

diretorio = "/content/drive/MyDrive/neg_train"
abrir_arquivos_em_ordem(diretorio, negative_dictionary)

Total de arquivos abertos com sucesso: 12500


In [ ]:
print(len(negative_dictionary))

2204735


In [ ]:
dictionary_size = 256
positive_dictionary = {chr(i): i for i in range(dictionary_size)}

diretorio = "/content/drive/MyDrive/pos_train"
abrir_arquivos_em_ordem(diretorio, positive_dictionary)

Total de arquivos abertos com sucesso: 12500


In [ ]:
print(len(positive_dictionary))

2283997


In [ ]:
erro = 0
acerto = 0

In [ ]:
diretorio_positivo = "/content/drive/MyDrive/pos"
erro, acerto = abrir_arquivos_em_ordem_teste(negative_dictionary, positive_dictionary, diretorio_positivo, "positivo", erro, acerto)

diretorio_negativo = "/content/drive/MyDrive/neg"
erro, acerto = abrir_arquivos_em_ordem_teste(negative_dictionary, positive_dictionary, diretorio_negativo, "negativo", erro, acerto)

print("Erros: ", erro)
print("Acertos: ", acerto)

Total de arquivos abertos com sucesso: 13311
Total de arquivos abertos com sucesso: 12500
Erros:  6354
Acertos:  19457


In [ ]:
taxa = 100 * acerto/(erro+acerto)
print("A taxa de acerto foi de: ", taxa, "%")

A taxa de acerto foi de:  75.3825888187207 %


# Comparativo com IA

##Tensorflow : https://www.tensorflow.org/tutorials/keras/text_classification_with_hub?hl=pt-br


In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

##Download do dataset do IMDB-Reviews : https://www.tensorflow.org/datasets/catalog/imdb_reviews?hl=pt-br

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:50%]', 'train[50%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT42693/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT42693/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT42693/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


##Varredura no dataset

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch#Os labels presentes nos arquivos de download: 0 para avaliações negativas e 1 para positivas

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

##Construindo o modelo

In [ ]:
#Para criação da rede neural é preciso fazer stack de layers, para isso três decisões são importantes:
#Como representar o texto;
#Quantos layers usar no modelo;
#Quantas hidden units usar para cada layer;
#Neste exemplo o "input data" são frases e os labels para predição são 0 ou 1
#Uma forma de representar os textos é converter em "embedding vectors"
#Por fim, o tensor flow utiliza de um módulo de texto pré-treinado

embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"#Módulo: nnlm-en-dim50
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

###Construindo o modelo completo

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


##Otimizador

In [ ]:
#Como se trata de um problema de "sentiment analysis", a função de Loss/error utiliza a BinaryCrossentropy para classificação
#Além do mais, utliza o otimizador "adam" que se adapta ao longo do processo de treinamento
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

##Treinamento

In [ ]:
#Para o treinamento, os dados recebem um embaralhamento e são divididos em 10 épocas de 512 batchs
#Isso para que o modelo treinado não aprenda a classificar por ordem dos arquivos estão dispostos
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
25/25 [==============================] - 57s 2s/step - loss: 0.6366 - accuracy: 0.5796 - val_loss: 0.5782 - val_accuracy: 0.6617
Epoch 2/10
25/25 [==============================] - 42s 2s/step - loss: 0.5097 - accuracy: 0.7352 - val_loss: 0.4830 - val_accuracy: 0.7530
Epoch 3/10
25/25 [==============================] - 41s 2s/step - loss: 0.3939 - accuracy: 0.8309 - val_loss: 0.4082 - val_accuracy: 0.8166
Epoch 4/10
25/25 [==============================] - 45s 2s/step - loss: 0.2987 - accuracy: 0.8876 - val_loss: 0.3610 - val_accuracy: 0.8370
Epoch 5/10
25/25 [==============================] - 46s 2s/step - loss: 0.2272 - accuracy: 0.9224 - val_loss: 0.3343 - val_accuracy: 0.8517
Epoch 6/10
25/25 [==============================] - 42s 2s/step - loss: 0.1733 - accuracy: 0.9472 - val_loss: 0.3206 - val_accuracy: 0.8563
Epoch 7/10
25/25 [==============================] - 50s 2s/step - loss: 0.1310 - accuracy: 0.9634 - val_loss: 0.3188 - val_accuracy: 0.8666
Epoch 8/10
25/25 [==

## Avaliação do Modelo

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 12s - loss: 0.3538 - accuracy: 0.8496 - 12s/epoch - 237ms/step
loss: 0.354
accuracy: 0.850


# Análise de Dados


In [17]:
from tabulate import tabulate

# Dados da tabela
headers = ['Tamanho do Dicionario', 'Dado de entrada', 'Tempo de Treinamento(min)']
index = ['Dicionario Negativo', 'Dicionario Positivo', 'IA']

data = []
data.append([2204735, 12500, 10])
data.append([2283997, 12500, 10])
data.append([48191433, 25000, 10])

# Adicionando o índice às linhas
table_data = [[index[i]] + row for i, row in enumerate(data)]

# Adicionando o título do índice na primeira célula da primeira lista
headers.insert(0, 'Treinamento')

# Exibindo a tabela utilizando o tabulate
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+---------------------+-------------------------+-------------------+-----------------------------+
| Treinamento         |   Tamanho do Dicionario |   Dado de entrada |   Tempo de Treinamento(min) |
+=====================+=========================+===================+=============================+
| Dicionario Negativo |                 2204735 |             12500 |                          10 |
+---------------------+-------------------------+-------------------+-----------------------------+
| Dicionario Positivo |                 2283997 |             12500 |                          10 |
+---------------------+-------------------------+-------------------+-----------------------------+
| IA                  |                48191433 |             25000 |                          10 |
+---------------------+-------------------------+-------------------+-----------------------------+


In [18]:
# Dados da tabela
headers = ['Taxa de Acerto(%)', 'Dado de entrada', 'Tempo de Teste(min)']
index = ['LZW', 'IA']

data = []
data.append([75.38, 25000, 35])
data.append([85.0, 25000, 10])

# Adicionando o índice às linhas
table_data = [[index[i]] + row for i, row in enumerate(data)]

# Adicionando o título do índice na primeira célula da primeira lista
headers.insert(0, 'Testes')

# Exibindo a tabela utilizando o tabulate
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+----------+---------------------+-------------------+-----------------------+
| Testes   |   Taxa de Acerto(%) |   Dado de entrada |   Tempo de Teste(min) |
+==========+=====================+===================+=======================+
| LZW      |               75.38 |             25000 |                    35 |
+----------+---------------------+-------------------+-----------------------+
| IA       |               85    |             25000 |                    10 |
+----------+---------------------+-------------------+-----------------------+
